In [8]:
!pip install -r lancedb-dev/requirements.txt 


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Basic pylance operations and inter-conversion between formats

In [18]:
import lance

import pandas as pd
import pyarrow as pa

# pandas -> pyarrow -> parquet
df = pd.DataFrame({"a": [5], "b": [10]})
tbl = pa.Table.from_pandas(df)

uri = "/tmp/test.parquet"
pa.dataset.write_dataset(tbl, uri, format='parquet', existing_data_behavior='overwrite_or_ignore')

# parquet -> lance
uri2 = "/tmp/test.lance"
parquet = pa.dataset.dataset(uri, format='parquet')
lance.write_dataset(parquet, uri2, mode='overwrite')
dataset = lance.dataset(uri2)
assert isinstance(dataset, pa.dataset.Dataset)

# lance -> pandas
dataset.to_table().to_pandas()

# lance -> duckdb
import duckdb
duckdb.query("SELECT * FROM dataset LIMIT 10").to_df()

,a,b
0,5,10
